# EDA

In [1]:
import pandas as pd
import altair as alt

In [3]:
clean_data = pd.read_csv("../data/data_transaction_time_fixed.csv")
clean_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 15 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Unnamed: 0                         100000 non-null  int64  
 1   Card Identifier                    100000 non-null  object 
 2   Transaction Date                   100000 non-null  object 
 3   Transaction Time                   97963 non-null   object 
 4   Risk Assessment                    99028 non-null   float64
 5   Payment Method                     100000 non-null  object 
 6   Transaction Value                  100000 non-null  float64
 7   Merchant Location                  100000 non-null  object 
 8   Card Present Status                100000 non-null  object 
 9   Chip Usage                         100000 non-null  object 
 10  Cross-border Transaction (Yes/No)  100000 non-null  object 
 11  Acquiring Institution ID           10000

In [4]:
clean_data.describe()

,Unnamed: 0,Risk Assessment,Transaction Value,Merchant Category Code (MCC)
count,100000.000000,99028.000000,100000.000000,99997.000000
mean,49999.500000,843.157905,71.130702,5807.915887
std,28867.657797,747.146439,296.637398,1061.918829
min,0.000000,2.000000,0.000000,742.000000
25%,24999.750000,352.000000,6.840000,5411.000000
50%,49999.500000,592.000000,19.230000,5812.000000
75%,74999.250000,1052.000000,50.000000,5942.000000
max,99999.000000,4932.000000,15000.000000,9999.000000


In [5]:
clean_data.head()

,Unnamed: 0,Card Identifier,Transaction Date,Transaction Time,Risk Assessment,Payment Method,Transaction Value,Merchant Location,Card Present Status,Chip Usage,Cross-border Transaction (Yes/No),Acquiring Institution ID,Merchant Identifier,Merchant Category Code (MCC),Fraud Indicator (Yes/No)
0,0,card 1,2023-05-11,16:22:14,362.0,Paypass - Contactless,13.98,USA,CP,Yes,No,acquirer 1,merchant 1,5812.0,No
1,1,card 2,2023-06-05,15:16:35,602.0,Online,24.64,USA,CNP,No,No,acquirer 2,merchant 2,4121.0,No
2,2,card 3,2023-06-05,11:57:40,482.0,Unknown,15.00,USA,CNP,No,No,acquirer 3,merchant 3,7211.0,No
3,3,card 4,2023-07-20,18:26:30,947.0,Online,30.56,USA,CNP,No,No,acquirer 1,merchant 4,5814.0,No
4,4,card 5,2023-03-03,19:03:11,1382.0,Magnetic Stripe,50.85,USA,CP,No,No,acquirer 4,merchant 1,5812.0,No


In [6]:
len(clean_data['Card Identifier'].unique())

7872

## Porportion of Fraudulent Transactions per Merchant

In [44]:
# get number of transactions per merchant
merchant_group = clean_data.groupby(by='Merchant Identifier')
merchant_group = merchant_group.size().reset_index(name="Number of Transactions")
merchant_group.head()

,Merchant Identifier,Number of Transactions
0,merchant 1,10968
1,merchant 10,256
2,merchant 100,4
3,merchant 1000,15
4,merchant 1001,11


In [36]:
# get number of fraudulent and non-fraudulent transactions per merchant
merchant_fraud = clean_data.groupby(by=['Merchant Identifier', 'Fraud Indicator (Yes/No)'])
merchant_fraud = merchant_fraud.size(
                                ).reset_index(name="Count"
                                ).pivot(index='Merchant Identifier', columns='Fraud Indicator (Yes/No)', values='Count'
                                ).reset_index(
                                ).fillna(0).rename_axis('', axis='columns'
                                ).rename(columns={'No':'Non-Fraudulent Transactions', 'Yes':'Fraudulent Transactions'})
merchant_fraud

,Merchant Identifier,Non-Fraudulent Transactions,Fraudulent Transactions
0,merchant 1,10956.0,12.0
1,merchant 10,256.0,0.0
2,merchant 100,4.0,0.0
3,merchant 1000,15.0,0.0
4,merchant 1001,11.0,0.0
...,...,...,...
2653,merchant 995,4.0,0.0
2654,merchant 996,5.0,0.0
2655,merchant 997,16.0,0.0
2656,merchant 998,9.0,0.0


In [47]:
# merge the two above data frames and sort by proportion of fraudulent transactions per merchant (ascending)
merchant_transactions = merchant_group.merge(merchant_fraud, left_on='Merchant Identifier', right_on='Merchant Identifier', how='outer')
merchant_transactions = merchant_transactions.assign(proportion_of_fraudulent_transactions=merchant_transactions['Fraudulent Transactions'] / merchant_transactions['Number of Transactions'])
merchant_transactions = merchant_transactions.sort_values(by='proportion_of_fraudulent_transactions', ascending=False)

# save the resulting data frame
merchant_transactions.to_csv("../data/eda_data_merchant_transaction_data.csv")

merchant_transactions

,Merchant Identifier,Number of Transactions,Non-Fraudulent Transactions,Fraudulent Transactions,proportion_of_fraudulent_transactions
1806,merchant 2624,1,0.0,1.0,1.0
2324,merchant 699,1,0.0,1.0,1.0
1275,merchant 2146,1,0.0,1.0,1.0
1187,merchant 2067,1,0.0,1.0,1.0
1515,merchant 2362,3,0.0,3.0,1.0
...,...,...,...,...,...
903,merchant 1811,1,1.0,0.0,0.0
904,merchant 1812,1,1.0,0.0,0.0
905,merchant 1813,1,1.0,0.0,0.0
906,merchant 1814,3,3.0,0.0,0.0


In [45]:
# viz the top 50 merchants with the most fraudulent transactions
top50_fraudsters = merchant_transactions.head(50)

plot_fraud_merchant = alt.Chart(top50_fraudsters).mark_bar().encode(
                        alt.X('Merchant Identifier', sort='-y'),
                        alt.Y('proportion_of_fraudulent_transactions')
)
plot_fraud_merchant

alt.Chart(...)